In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [2]:
# Define the function to load the dataset
def load_path(path, part):
    dataset = []
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        if os.path.isdir(folder_path):
            for body in os.listdir(folder_path):
                if body == part:
                    body_part = body
                    body_path = os.path.join(folder_path, body)
                    for patient_id in os.listdir(body_path):
                        patient_path = os.path.join(body_path, patient_id)
                        for lab in os.listdir(patient_path):
                            label = 'fractured' if lab.split('_')[-1] == 'positive' else 'normal'
                            lab_path = os.path.join(patient_path, lab)
                            for img in os.listdir(lab_path):
                                img_path = os.path.join(lab_path, img)
                                dataset.append({
                                    'body_part': body_part,
                                    'patient_id': patient_id,
                                    'label': label,
                                    'image_path': img_path
                                })
    return dataset


In [3]:
def prepare_data(part):
    image_dir = './dataset/'
    data = load_path(image_dir, part)
    labels = [row['label'] for row in data]
    filepaths = [row['image_path'] for row in data]

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    train_df, test_df = train_test_split(images, train_size=0.9, shuffle=True, random_state=1)

    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        horizontal_flip=True,
        preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
        validation_split=0.2
    )

    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.resnet50.preprocess_input
    )

    train_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=64,
        shuffle=True,
        seed=42,
        subset='training'
    )

    val_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=64,
        shuffle=True,
        seed=42,
        subset='validation'
    )

    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )

    return train_images, val_images, test_images

In [4]:
def create_model():
    pretrained_model = tf.keras.applications.ResNet50(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg'
    )

    pretrained_model.trainable = False

    inputs = pretrained_model.input
    x = pretrained_model.output
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)  # Add dropout for regularization
    x = tf.keras.layers.BatchNormalization()(x)  # Add batch normalization
    x = tf.keras.layers.Dense(50, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)  # Add dropout for regularization
    outputs = tf.keras.layers.Dense(2, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [5]:
def train_model(part, train_images, val_images, test_images):
    model = create_model()

    print("-------Training " + part + "-------")

    # Define callbacks
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(os.getcwd(), f"weights/ResNet50_{part}_frac.h5"),
        monitor='val_loss',
        save_best_only=True
    )

    reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=2,
        min_lr=0.0001
    )

    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=3, restore_best_weights=True
    )

    # Train the model
    history = model.fit(
        train_images,
        validation_data=val_images,
        epochs=25,
        callbacks=[checkpoint_callback, reduce_lr_callback, early_stopping_callback]
    )

    # Evaluate the model
    results = model.evaluate(test_images, verbose=0)
    print(part + " Results:")
    print(results)
    print(f"Test Accuracy: {np.round(results[1] * 100, 2)}%")

    # Plot accuracy and loss
    os.makedirs(f"./plots/FractureDetection/{part}", exist_ok=True)

    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(f"./plots/FractureDetection/{part}/_Accuracy.jpeg")
    plt.clf()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(f"./plots/FractureDetection/{part}/_Loss.jpeg")
    plt.clf()

In [6]:
categories_parts = ["Elbow", "Hand", "Shoulder"]

for part in categories_parts:
    train_images, val_images, test_images = prepare_data(part)
    train_model(part, train_images, val_images, test_images)

Found 3885 validated image filenames belonging to 2 classes.
Found 971 validated image filenames belonging to 2 classes.
Found 540 validated image filenames belonging to 2 classes.
-------Training Elbow-------
Epoch 1/25
61/61 [==============================] - 35s 261ms/step - loss: 0.7899 - accuracy: 0.5709 - val_loss: 0.6155 - val_accuracy: 0.6643 - lr: 1.0000e-04
Epoch 2/25
61/61 [==============================] - 12s 196ms/step - loss: 0.6593 - accuracy: 0.6541 - val_loss: 0.5708 - val_accuracy: 0.7291 - lr: 1.0000e-04
Epoch 3/25
61/61 [==============================] - 12s 197ms/step - loss: 0.6089 - accuracy: 0.6986 - val_loss: 0.5466 - val_accuracy: 0.7559 - lr: 1.0000e-04
Epoch 4/25
61/61 [==============================] - 12s 202ms/step - loss: 0.5859 - accuracy: 0.6983 - val_loss: 0.5286 - val_accuracy: 0.7621 - lr: 1.0000e-04
Epoch 5/25
61/61 [==============================] - 12s 193ms/step - loss: 0.5525 - accuracy: 0.7351 - val_loss: 0.5262 - val_accuracy: 0.7621 - lr: 1

<Figure size 640x480 with 0 Axes>